참고: https://github.com/kairess/mental-health-chatbot

In [ ]:
!pip install sentence_transformers

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('mode.chained_assignment',  None)

# SentenceBERT 모델 로드

In [ ]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask') #PLM

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = model.encode(sentences)

print(embeddings)

# 데이터셋 로드

웰니스 대화 스크립트 데이터셋

https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-006

다운로드 링크

https://drive.google.com/uc?export=download&id=1MiMBWmI9-7uBZx6G7O3Vpae99ZoPfxYx


In [ ]:
df = pd.read_csv('wellness_dataset_original.csv')

df.head(20)

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN
5,감정/감정조절이상,작년 가을부터 감정조절이 잘 안 되는 거 같아.,NaN
6,감정/감정조절이상,예전에 비해서 인내심이 너무 짧아진 거 같아.,NaN
7,감정/감정조절이상,더 이상 혼자서는 감정조절을 못하겠어.,NaN
8,감정/감정조절이상,점점 나 자신을 컨트롤하지 못하는 기분이야.,NaN
9,감정/감정조절이상,나도 이러기 싫은데 내 마음대로 안돼.,NaN


# 전처리

In [ ]:
df = df[~df['챗봇'].isna()]

df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.


In [ ]:
df.loc[0, '유저']

'제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.'

In [ ]:
model.encode(df.loc[0, '유저'])

array([-4.80606377e-01, -2.94869184e-01,  4.37900275e-01, -6.40137970e-01,
        3.28670815e-02, -3.42647403e-01, -5.47482446e-02,  1.73052941e-02,
       -4.08220708e-01, -5.06033957e-01, -1.68733254e-01, -3.98677021e-01,
       -1.24776624e-01, -9.71540287e-02, -1.65286660e-01,  5.72618330e-03,
        6.13691397e-02, -1.91312179e-01,  2.53917515e-01, -5.85019767e-01,
       -2.84425884e-01, -2.32034951e-01, -3.27080727e-01,  6.72976151e-02,
       -1.66169229e-05, -4.72336233e-01, -3.60021770e-01,  2.91879624e-01,
       -6.63861573e-01, -3.10574740e-01,  5.79525113e-01, -3.11722845e-01,
        1.47696631e-02, -2.12172866e-01,  2.22058222e-01, -1.73828915e-01,
       -3.78458172e-01, -4.20398444e-01, -2.38218769e-01,  6.38703331e-02,
       -1.15304030e-01, -2.44564295e-01, -5.00228584e-01,  1.68355241e-01,
       -6.58359945e-01, -8.91941965e-01, -6.26957119e-01, -3.21965545e-01,
       -7.05358684e-01,  3.71447355e-01, -5.45803368e-01,  7.76298046e-02,
        1.09864354e-01,  

# 유저 대화내용 인코딩

In [ ]:
df['embedding'] = pd.Series([[]] * len(df)) # dummy

df['embedding'] = df['유저'].map(lambda x: list(model.encode(x)))

df.head()

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060638, -0.29486918, 0.43790027, -0.6401..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561574, -0.14506255, 0.2949032, -0.673949..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652001, -0.081268296, 1.0945569, 0.105792..."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.7679057, 0.4652075, 0.5285071, -0.5076045,..."
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277719, -0.37413925, 0.04053233, -0.8624..."


In [ ]:
df.to_csv('wellness_dataset.csv', index=False)

# 간단한 챗봇

In [ ]:
text = '요즘 머리가 아프고 너무 힘들어'

embedding = model.encode(text)

In [ ]:
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head(20)

In [ ]:
answer = df.loc[df['distance'].idxmax()]

print('구분', answer['구분'])
print('유사한 질문', answer['유저'])
print('챗봇 답변', answer['챗봇'])
print('유사도', answer['distance'])

In [ ]:
# 챗봇 구현
print("그만하고 싶으시면 '그만'이라고 입력하세요.")
while 1:
    text=input("사용자: ")
    if text=="그만": print("안녕히 가세요."); break
    embedding = model.encode(text)
    df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
    answer = df.loc[df['distance'].idxmax()]
    answer=answer["챗봇"]
    print(f"챗봇: {answer}")